# 01B — Hello JSON (Refresher) → Getting to GeoJSON 🗺️

This notebook is a **JSON warm-up**.

### Goals
By the end of this section you should be able to:
- ~~Load JSON from a string / file~~
- Traverse JSON-like data structures (nested `dict` + `list`)
- ~~Recognize *the shape* of GeoJSON (FeatureCollection → Features → Geometry/Properties)~~
- ~~Understand why humans struggle switching between dicts and lists (even when it’s “just a dictionary”) 😅~~


In [3]:
import json
from rich import print  # better library than pprint

print("Ready ✅")


Ready ✅

## 01B.1 A small “generic JSON” example

- We’ll start with a plain JSON object that contains:
  - simple fields
  - a list of strings
  - a list of objects (list of dicts)
  - a nested dict

- We first must load a dictionary

In [8]:
with open('./Resources/Data/initial_dict.json') as f:
    data = json.load(f)

### 01B.2 Iterate a `dict` (key/value pairs)

Common patterns:
- `for key in d:`  (keys only)
- `for key, value in d.items():`  (**keys + values**)


In [ ]:
for k, v in data.items():
    print(f"{k:10s} -> {type(v).__name__}")


course     -> str

week       -> int

topics     -> list

instructor -> dict

students   -> list

### 01B.3 Iterate a `list`

The `topics` field is a list of strings.


In [9]:
topics = data["topics"]
print(type(topics).__name__, topics)

for t in topics:
    print("topic:", t)


list
['json', 'geojson', 'files']

topic: json

topic: geojson

topic: files

### 01B.4 Iterate a “list of dicts”

`students` is a list, but each element is a dict.

So you do:
1) loop the list  
2) inside the loop, access dict keys


In [ ]:
students = data["students"]
for s in students:
    print(s["name"], "->", s["year"])


## 01B.5 A tiny helper: “what am I holding right now?”

When you’re lost, print:
- `type(x)`
- keys (if dict)
- length (if list)

Make this a habit and JSON stops being spooky.


In [10]:
def describe(x, label="value"):
    print(f"{label}: type={type(x).__name__}")
    if isinstance(x, dict):
        print("  keys:", list(x.keys()))
    elif isinstance(x, list):
        print("  len:", len(x))
        if x:
            print("  first element type:", type(x[0]).__name__)

describe(data, "data")
describe(data["students"], "students")
describe(data["students"][0], "students[0]")


data: type=dict

keys:
['course', 'week', 'topics', 'instructor', 'students']

students: type=list

len: 3

first element type: dict

students[0]: type=dict

keys:
['name', 'year']

## 01B.6  (Optional but useful) Recursive traversal

Sometimes you don’t know the shape ahead of time. A simple recursive walker can:
- visit every dict key
- visit every list element
- show you the “path” down into the structure

This is **not** magic — it’s just “if dict do dict things; if list do list things.”


In [11]:
def walk(x, path="$", max_items=5):
    """Print a compact view of a nested dict/list structure."""
    if isinstance(x, dict):
        print(f"{path} (dict) keys={len(x)}")
        for i, (k, v) in enumerate(x.items()):
            if i >= max_items:
                print(f"{path} ... ({len(x)-max_items} more keys)")
                break
            walk(v, f"{path}.{k}", max_items=max_items)

    elif isinstance(x, list):
        print(f"{path} (list) len={len(x)}")
        for i, v in enumerate(x[:max_items]):
            walk(v, f"{path}[{i}]", max_items=max_items)
        if len(x) > max_items:
            print(f"{path} ... ({len(x)-max_items} more items)")

    else:
        print(f"{path} ({type(x).__name__}) = {x!r}")

walk(data)


$ (dict) keys=5

$.course (str) = 'Spatial Data & Mapping'

$.week (int) = 1

$.topics (list) len=3

$.topics[0] (str) = 'json'

...

$.students[1].name (str) = 'Linus'

$.students[1].year (str) = 'Junior'

$.students[2] (dict) keys=2

$.students[2].name (str) = 'Grace'

$.students[2].year (str) = 'Senior'

## 01B.7 GeoJSON teaser (do **not** panic)

GeoJSON is “just JSON” with a convention:
- Top level is often a `FeatureCollection`
- It has a `features` list
- Each feature has:
  - `geometry` (shape + coordinates)
  - `properties` (your attributes / labels / metadata)

We’ll barely discuss it here — this is a segue into the next lesson.


In [15]:
import json 

geojson_text = r'''{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {"type": "Point", "coordinates": [-98.5263, 33.8719]},
      "properties": {"name": "Coffee Shop"}
    },
    {
      "type": "Feature",
      "geometry": {"type": "Point", "coordinates": [-98.5281, 33.8725]},
      "properties": {"name": "School"}
    },
    {
      "type": "Feature",
      "geometry": {"type": "Point", "coordinates": [-98.5302, 33.8731]},
      "properties": {"name": "Bookstore"}
    },
    {
      "type": "Feature",
      "geometry": {
        "type": "MultiLineString",
        "coordinates": [
          [
            [-98.5263, 33.8719],
            [-98.5281, 33.8725],
            [-98.5302, 33.8731]
          ]
        ]
      },
      "properties": {"name": "Route from Coffee to Bookstore"}
    },
    {
      "type": "Feature",
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [-98.531, 33.871],
            [-98.531, 33.874],
            [-98.525, 33.874],
            [-98.525, 33.871],
            [-98.531, 33.871]
          ]
        ]
      },
      "properties": {"name": "Bounding Box"}
    }
  ]
}'''
geo = json.loads(geojson_text)

describe(geo, "geo")
describe(geo["features"], "geo['features']")
describe(geo["features"][0], "first feature")


geo: type=dict

keys:
['type', 'features']

geo['features']: type=list

len: 5

first element type: dict

first feature: type=dict

keys:
['type', 'geometry', 'properties']

### 01B.8 The “shape” of GeoJSON (the whole trick)

- `geo` is a **dict**
- `geo["features"]` is a **list**
- each `feature` is a **dict**
- `feature["geometry"]` is a **dict**
- `feature["geometry"]["coordinates"]` is a **list** (nesting depth depends on geometry type)

Nesting depth patterns:
- **Point** → `[lon, lat]`
- **LineString** → `[[lon, lat], [lon, lat], ...]`
- **MultiLineString** → `[[[lon, lat], ...], [[lon, lat], ...], ...]`
- **Polygon** → `[[[lon, lat], ...]]]` (rings)
- **MultiPolygon** → `[[[[lon, lat], ...]]]]` (multiple polygons)


In [ ]:
from collections import Counter

types = Counter(f["geometry"]["type"] for f in geo["features"])
print("Geometry types:", dict(types))

for f in geo["features"]:
    g = f["geometry"]
    print(f"- {g['type']:<15s}  name={f['properties'].get('name')}")


### 01B.9 Coordinates: quick previews (nesting-level problem)

We won't solve everything here — just enough to recognize patterns.

Next lesson: we’ll write functions that handle each geometry type correctly (and catch common mistakes).


In [ ]:
def coord_preview(geom, n=2):
    coords = geom["coordinates"]
    t = geom["type"]
    print(f"\n{t}:")
    if t == "Point":
        print("  coords =", coords)
    else:
        # show a compact preview without dumping everything
        pprint(coords[:n])

for feat in geo["features"]:
    coord_preview(feat["geometry"])


## 01B.10  Mini check (practice)

### I want you to:

In a new code cell: 

1. Create a file in `./Resources/Data` called `geodata.geojson` with the geojson file from above so you can open and process it.
   
2. Open that file programmatically and: 
   
   1. Print the name of every feature.
   2. Count how many Points vs non-Points exist.
   3. For each Point, unpack `(lon, lat)` and print them.
   4. For the Polygon, print the number of vertices in its outer ring.

If any of those feel hard, that’s the *point* — it means you’re learning the traversal patterns.

- Save your output to a file by writing it to: `./Resources/Data/01B_output.pdf` and ensure it is saved with nice and readable format indicating each of the specified items (1-4).
  
- Ensure everything is commented, and labeled clearly. 
   
- Save notebook to your `Assignments Completed` folder in a the following format:

```sh
       Assignments_Completed
      ├──  02
      │   ├──  01B
      │   │   ├──  01B_notebook.ipynb  # your code
      │   │   ├──  output.pdf # your formatted output
      │   └── 󰂺 README.md
      └── 󰂺 README.md
```